In [52]:
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# from sklearn.model_selection import KFold # 交差検証法に関する関数

import warnings
warnings.simplefilter('ignore')


## データの読み込み

In [83]:
# データの読み込み
# df_org = pd.read_csv('./data/input_01.csv')
# df_org = pd.read_csv('./data/input_02.csv')
df_org = pd.read_csv('./data/input_03.csv')


In [98]:
# データ量が多いのでコーディング中はサンプリング
# df = df_org.sample(10000, replace=True)
df = df_org

## データの準備

In [99]:
# 数値エンコーディング（GBDT以外の場合は必須）
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df = df.apply(encoder.fit_transform)

In [100]:
y = df[['state']]
# x_cols = ['category', 'main_category', 'currency', 'country', 'period', 'n_words', 'log_goal']
# x_cols = ['class', 'currency', 'country', 'period', 'n_words', 'log_goal']
x_cols = ['class', 'main_category', 'currency', 'country', 'period', 'n_words', 'log_goal']
x = df[x_cols]

In [101]:
# y.shape

In [102]:
# x.head()

## モデル学習（単体）

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

# 訓練データとテストデータを8:2に分割
# ※評価を揃えるため df 以外の引数は変更しないこと
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1234)

# パラメータ
RANDOM_STATE = 10
num_round = 1000
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=TEST_SIZE,random_state=RANDOM_STATE)

# lgb_train = lgb.Dataset(x_train, y_train)
# lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

In [104]:
# 訓練データに対する予測、正答率
# 学習の実行
clf = SGDClassifier(loss='log', max_iter=1000, tol=1e-3, random_state=seed)
clf.fit(x_train, y_train)

# 推論
y_pred = clf.predict(x_train)
# print(y_train)
acc_train = accuracy_score(y_true=y_train, y_pred=y_pred)
# print(f"{itr:02}train_訓練データの正答率: {100 * acc_train:.2f} %")
print(f"訓練データの正答率: {100 * acc_train:.3f} %")

訓練データの正答率: 61.443 %


## モデル学習（交差検証）

In [105]:
from sklearn.model_selection import KFold
n_split = 5 # グループ数を設定（今回は5分割）
seed = 1234
# cross_valid_mae = 0
split_num = 1
kf = KFold(n_splits=n_split, shuffle=True, random_state=seed)

In [106]:
from sklearn.metrics import accuracy_score #, precision_score, recall_score, f1_score
from sklearn.linear_model import SGDClassifier

scores_train = []
scores_valid = []
models = []
itr = 1

for train_idx, valid_idx in kf.split(x, y):
    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_valid, y_valid = x.iloc[valid_idx], y.iloc[valid_idx]
    
    # 学習の実行
    clf = SGDClassifier(loss='log', max_iter=1000, tol=1e-3, random_state=seed)
    clf.fit(x_train, y_train)
    
    # # 訓練データに対する予測、正答率
    # y_pred = clf.predict(x_train)
    # acc_train = accuracy_score(y_true=y_train, y_pred=y_pred)
    # print(f"{itr:02}train_訓練データの正答率: {100 * acc_train:.2f} %")
    # scores_train.append(acc_train)  
    
    # 検証データに対する予測
    y_pred = clf.predict(x_valid)
    acc_valid = accuracy_score(y_true=y_valid, y_pred=y_pred)
    print(f"{itr:02}valid_検証データの正答率: {100 * acc_valid:.2f} %")    
    

    scores_valid.append(acc_valid)
    models.append(clf)

    itr += 1

01valid_検証データの正答率: 61.56 %
02valid_検証データの正答率: 62.06 %
03valid_検証データの正答率: 60.49 %
04valid_検証データの正答率: 61.62 %
05valid_検証データの正答率: 61.73 %


In [107]:
# 各foldのスコアの平均
print(np.mean(scores_train))
print(np.mean(scores_valid))

nan
0.6149212331348458


## 推論

In [108]:
y_preds = np.zeros(len(x_test))

for model in models:
    y_pred = model.predict(x_test)
    y_pred = y_pred.round(0)
    y_preds += y_pred / len(models)
    
y_preds = y_preds.round(0)
score_test = accuracy_score(y_test, y_preds)
print(f"test_accuracy_score: {score_test:.4f} %")    

test_accuracy_score: 0.6212 %


### input_01.csv
- 訓練データの正答率（fold平均）：0.621925
- 検証データの正答率（fold平均）：0.6247

### input_02.csv（class, main_category）
- 訓練データの正答率（fold平均）：0.553375
- 検証データの正答率（fold平均）：0.55099

### input_02.csv（class）
- 訓練データの正答率（fold平均）：0.572375
- 検証データの正答率（fold平均）：0.57800

## テストデータに対する予測
本来はte